In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [5]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

[Errno 2] No such file or directory: 'drive/MyDrive/Dacon/LDI'
/content/drive/MyDrive/Dacon/LDI


In [6]:
test_err_data = pd.read_csv('./dataset/test_err_data.csv',thousands = ',')
test_quality_data = pd.read_csv('./dataset/test_quality_data.csv',thousands = ',')

In [8]:
df_ted = test_err_data.drop_duplicates()
df_tqd = test_quality_data.drop_duplicates()

In [9]:
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'connection timeout'),'errtype'] = '23_ct'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'active'),'errtype'] = '23_as'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'standby'),'errtype'] = '23_ac'
df_ted.loc[(df_ted['errtype'] == 31) & (df_ted['errcode'] == '0'),'errtype'] = '31_0'
df_ted.loc[(df_ted['errtype'] == 40) & (df_ted['errcode'] == '0'),'errtype'] = '40_0'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
for i in range(0,13):
    qstr = 'quality_'+str(i)
    df_tqd.loc[df_tqd[qstr] != -1,qstr] = 0
    df_tqd.loc[df_tqd[qstr] == -1,qstr] = 1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [11]:
df_ted = df_ted.drop('fwver',axis=1)
df_ted = df_ted.drop('model_nm',axis=1)
df_tqd = df_tqd.drop('fwver',axis=1)
df_ted = df_ted.drop('errcode',axis=1)

In [12]:
times = list(df_ted['time'])

In [13]:
times = list(df_ted['time'])

for i in range(len(times)):
    try:
        times[i] = int(str(times[i])[4:8])
    except ValueError:
        print(i)
    

df_ted['time'] = times

df_ted.head()

,user_id,time,errtype
0,30000,1101,31
1,30000,1101,33
2,30000,1101,15
3,30000,1101,22
4,30000,1101,11


In [14]:
print(len(df_ted['time'].unique()))
print(len(df_ted['errtype'].unique()))

39
46


In [15]:
df_err = df_ted[['user_id','errtype']].values
df_err

array([[30000, 31],
       [30000, 33],
       [30000, 15],
       ...,
       [44998, 15],
       [44998, 16],
       [44998, '31_0']], dtype=object)

In [18]:
from tqdm import tqdm

df_x = np.zeros((14999,60))
df_err = df_ted[['user_id','errtype']].values
for pi,err in tqdm(df_err):
    if err == '23_ct':
        err = 43
    elif err == '23_as':
        err = 44
    elif err == '23_ac':
        err = 45
    elif err == '31_0':
        err = 46
    elif err == '40_0':
        err = 47
    df_x[int(pi)-30000,int(err)-1] += 1

for i in range(0,13):
    a = "quality_"+ str(i)
    df_qua = df_tqd[['user_id',a]].values
    for pi, qua in tqdm(df_qua):
        df_x[int(pi)-30000,i + 47] += 1

df_err.shape

100%|██████████| 243829/243829 [00:00<00:00, 458018.88it/s]


(15527225, 2)

In [19]:
df_x.shape

(14999, 60)

In [20]:
df_x[:10]

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 4.400e+01, 1.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.600e+01, 1.600e+01,
        0.000e+00, 3.000e+00, 6.600e+01, 3.700e+01, 9.000e+00, 3.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 9.830e+02, 5.600e+01, 5.000e+00,
        2.000e+00, 3.200e+01, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        4.300e+01, 2.000e+00, 1.600e+01, 0.000e+00, 1.000e+00, 3.000e+00,
        3.000e+00, 0.000e+00, 2.000e+00, 4.600e+01, 0.000e+00, 2.000e+00,
        1.107e+03, 1.410e+02, 4.000e+00, 4.000e+01, 4.600e+01, 2.000e+00,
        2.000e+00, 2.000e+00, 2.000e+00, 2.000e+00, 2.000e+00, 2.000e+00,
        2.000e+00, 2.000e+00, 2.000e+00, 2.000e+00, 2.000e+00, 2.000e+00],
       [0.000e+00, 0.000e+00, 3.000e+00, 9.800e+01, 3.000e+00, 4.000e+00,
        4.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.200e+01, 1.600e+01,
        4.000e+00, 6.000e+00, 5.300e+01, 5.500e+01, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+

In [21]:
import pickle
with open('testdata.txt', 'wb') as f:
    pickle.dump(df_x, f)